In [14]:
import pandas as pd
import json as json

In [16]:
def loadFiles(version, arena):
    file_path     = "../data/"
    list_of_files = []
    arena_list    = ["arena-1", "arena-2", "arena-3", "arena-4", "arena-5", "arena-6", "arena-7", "arena-8", "arena-9",]
    file_path = getVersionPath(version, file_path)
    if arena in arena_list:   
        file_path        += "/" + arena + "/"
        encounter_pokemon = file_path + "EncounterablePokemon.csv"
        pokemon_of_leader = file_path + "LeaderPokemon.csv"
        arena_leader = file_path + "ArenaLeader.csv"
        move_of_in        = file_path + "MovesOfInterest.csv"
        list_of_files.append(encounter_pokemon)
        list_of_files.append(pokemon_of_leader)
        list_of_files.append(arena_leader)
        list_of_files.append(move_of_in)  
    else:
        print("[-] Arena not found...")
    return list_of_files

def getVersionPath(version, file_path):
    if version == "red":
        file_path += "red"
        return file_path
    elif version == "blue":
        file_path += "blue"
        return file_path

In [17]:
def loadStones(arena):
    #["moon-stone", "fire-stone", "thunder-stone", "water-stone", "leaf-stone"]
    ## TODO: add arena stone data
    list_of_stones = []
    
    arena1_stones = []
    arena2_stones =  arena1_stones + ["moon-stone"]
    arena3_stones = arena2_stones + []
    
    if arena == "arena-1":
        pass
    elif arena == "arena-2":
        for stone in arena2_stones:  
            list_of_stones.append(stone)
    elif arena == "arena-3":
        for stone in arena3_stones:
            list_of_stones.append(stone)
    return list_of_stones

In [274]:
# STATIC CSV FILES
pokemonTable = pd.read_csv('../Data/Pokemon.csv')
pokemonTypeTable = pd.read_csv('../Data/PokemonTypeChart.csv')

# SETUP ARENA-1
ARENA = 'arena-1'
GAME = 'blue'
csvPaths = loadFiles(GAME, ARENA)

encounterable_pokemonTable = pd.read_csv(csvPaths[0])

# [name, moves, encounter, originIfEvo, movesByVM, movesByTM]
encounterable_pokemonParsed = []

for row in encounterable_pokemonTable.values.tolist():
    row = row[1:]
    moves = row[1]
    moves = moves.replace("'", "\"") 
    moves = json.loads(moves)
    row[1] = moves
    
    VMmoves = row[4]
    VMmoves = VMmoves.replace("'", "\"") 
    VMmoves = json.loads(VMmoves)
    row[4] = VMmoves
    
    TMmoves = row[5]
    TMmoves = TMmoves.replace("'", "\"") 
    TMmoves = json.loads(TMmoves)
    row[5] = TMmoves
   
    encounterable_pokemonParsed.append(row)
    
leader_pokemon =  pd.read_csv(csvPaths[1])

## um moves von leaderPokemon zu parsen
#test = leader_pokemon.loc[leader_pokemon['name']=='geodude']['moves'].values.tolist()[0].replace("'", "\"") 
#json.loads(test)

arena_leader = pd.read_csv(csvPaths[2])
arena_leader_parsed = []
for row in arena_leader.values.tolist(): 
    moves = row[3]
    moves = moves.replace("'", "\"") 
    moves = json.loads(moves)
    row[3] = moves
    arena_leader_parsed.append(row[1:])

moves_of_interest = pd.read_csv(csvPaths[3])

## extracted Variables from DATAFRAME
AVERAGE_LEVEL = arena_leader['averageLevel'][0]

## Variables for each Arena
arena1_Options = loadStones(ARENA)

## get leaderPokemonData 
# [stats, moves]
leaderPokemonData = {}
for leaderPokemon in arena_leader_parsed[0][2]:
    moves = leader_pokemon.loc[leader_pokemon['name']==leaderPokemon]['moves'].values.tolist()[0].replace("'", "\"") 
    moves = json.loads(moves)
    moveDict = {}
    for move in moves:
        moveData = moves_of_interest.loc[moves_of_interest['name']==move]
        moveDict[move] = moveData
    leaderPokemonData[leaderPokemon] = [ extractPokemonStatsFromTable(leaderPokemon), moveDict] 


In [233]:
# extracts all data of interest for the enemy leader pokemon
def extractPokemonStatsFromTable(pokemonName):
    pokeRow = pokemonTable.loc[pokemonTable['Name'] == pokemonName]
    return pokeRow

In [192]:
# spuckt liste von Pokemon aus welche zur verf. stehen

# @para1: encounterable_pokemon - liste von antreffbaren pokemon :: [index,name,encounterOrEvolutionTrigger,...]
# @para2: averageLevel - averageLevel des ArenaLeiters
# @para3: stones_available - Die Entw. Steine welche zur verfuegung stehen
# @return: poss_team_member - liste von pokemon welche mit dem averageLevel und den vorhanden Steinen erreichbar sind :: [index,name,encounterOrEvolutionTrigger,...]
def possPokemonArenaNLeaderM(encounterable_pokemon, averageLevel, stones_available):
    
    encounterable_pokemon_available = []
    for pokemon in encounterable_pokemon:
        encounterOrEvolutionTrigger = pokemon[2]
        if encounterOrEvolutionTrigger.isnumeric() and int(encounterOrEvolutionTrigger) <= averageLevel:
            encounterable_pokemon_available.append(pokemon)
        elif encounterOrEvolutionTrigger == 'walk' or encounterOrEvolutionTrigger == 'starter' or encounterOrEvolutionTrigger == 'special' or encounterOrEvolutionTrigger == 'trade':
            encounterable_pokemon_available.append(pokemon)
        elif encounterOrEvolutionTrigger in stones_available:
            encounterable_pokemon_available.append(pokemon)
    return encounterable_pokemon_available

In [351]:
possiblePokemon = possPokemonArenaNLeaderM(encounterable_pokemonParsed, AVERAGE_LEVEL, arena1_Options)

In [164]:
def extractMoveData(moveName):
    return moves_of_interest.loc[moves_of_interest['name']==moveName]

In [204]:
# extracts available move names from pokemon list item
def calcAvailableMoveNamesForPokemon(ourPokemonAndMoves, averageLevel):
    moves = []
    # LEVEL UP MOVES
    for move in ourPokemonAndMoves[1]:
        if move['levelLearnedAt'] <= averageLevel:
            moves.append(move['name'])
    # VM MOVES
    for move in ourPokemonAndMoves[4]:
        moves.append(move)
    # TM MOVES
    for move in ourPokemonAndMoves[5]:
        moves.append(move)
    return moves

In [383]:
def getMoveData(pokemonMoveNames):
    moveData = []
    for move in pokemonMoveNames:
        moveData.append(extractMoveData(move))
    return moveData

In [286]:
def retreiveMoveEffectiveness(moveType, vsMoveType):
    return pokemonTypeTable.loc[pokemonTypeTable['type'] == moveType][vsMoveType].values[0]

In [294]:
avail_moves = calcAvailableMoveNamesForPokemon(encounterable_pokemonParsed[0], AVERAGE_LEVEL)

testPokeObject = createOurPokemonObject(encounterable_pokemonParsed[0][0], avail_moves)

In [409]:
def calcEffectiveMovesVs(ourPokeMoves, theirPoke):
    theirType1 = theirPoke[0]['Type 1'].values[0].lower()
    theirType2 = theirPoke[0]['Type 2'].values[0].lower()
    effectivenessList = []
    for move in ourPokeMoves:
        moveType = move['moveType'].values[0]
        moveName =  move['name'].values[0]
        effective = retreiveMoveEffectiveness(moveType, theirType1)
        effectivenessList.append({'name':moveName, 'effective':effective})
    effectivenessList = sorted(effectivenessList, key=lambda d: d['effective'],reverse=True) 
    return(effectivenessList)

In [410]:
calcEffectiveMovesVs(testPokeObject[2], leaderPokemonData['onix'])

[{'name': 'sand-attack', 'effective': 2.0},
 {'name': 'gust', 'effective': 0.5},
 {'name': 'quick-attack', 'effective': 0.5}]

In [417]:
def calcAllPokemonVsArenaPokemon(possiblePokemons, arenaPokemon, averageLevel):
    pokemonWithData = []
    for pokemon in possiblePokemon:
        pokemonName = pokemon[0]
        availableMoves = calcAvailableMoveNamesForPokemon(pokemon, averageLevel)
        availableMove_Data = getMoveData(availableMoves)
        pokeBaseStats = extractPokemonStatsFromTable(pokemonName)
        effectiveMoves = calcEffectiveMovesVs(availableMove_Data, arenaPokemon)
        pokemonWithData.append({'name':pokemonName, 'moves':effectiveMoves, 'stats':pokeBaseStats})
    return (pokemonWithData)


In [418]:
calcAllPokemonVsArenaPokemon(encounterable_pokemonParsed, leaderPokemonData['geodude'], AVERAGE_LEVEL)

[{'name': 'pidgey',
  'moves': [{'name': 'sand-attack', 'effective': 2.0},
   {'name': 'gust', 'effective': 0.5},
   {'name': 'quick-attack', 'effective': 0.5}],
  'stats':     Unnamed: 0   #    Name  Type 1  Type 2  Total  HP  Attack  Defense  \
  20          20  16  pidgey  Normal  Flying    251  40      45       40   
  
      Sp. Atk  Sp. Def  Speed  Generation  Legendary  
  20       35       35     56           1      False  },
 {'name': 'rattata',
  'moves': [{'name': 'tackle', 'effective': 0.5},
   {'name': 'tail-whip', 'effective': 0.5},
   {'name': 'quick-attack', 'effective': 0.5}],
  'stats':     Unnamed: 0   #     Name  Type 1 Type 2  Total  HP  Attack  Defense  \
  24          24  19  rattata  Normal    NaN    253  30      56       35   
  
      Sp. Atk  Sp. Def  Speed  Generation  Legendary  
  24       25       35     72           1      False  },
 {'name': 'caterpie',
  'moves': [{'name': 'string-shot', 'effective': 1.0},
   {'name': 'tackle', 'effective': 0.5}],
  's